In [1]:
from invert_embeddings import *
from network_stats import *
from predict import *

In [2]:
# Dataset
network_name = "sbm4.mat"
# Optimization Rank
rank = 128
# Window Size (T)
window = 10

## Load Network and Low-dimensional Embedding

In [3]:
N = Network( )
# Load network
N.loadNetwork( network_name, True )
# Get largest connected component
N.standardize()
# PMI or PPMI matrix
skip_max = False
# Calculate (P)PMI matrix
N.netmf_embedding( window, skip_max )
# And low-rank dimensional embedding
N.low_rank_embedding( rank )

/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


## Optimization Step

In [ ]:
# Instantiate class
# Parameters: Adjacency Matrix, Low-Dimensional Embedding, Filename, (Deprecated Argument), Rank
P = Optimizer( N.getAdjacency(), N.get_LR_embedding(), network_name, 1, str(rank) )
P.learnNetwork() 

## Network Reconstruction

In [5]:
# Binarization method
method = "coin_toss"
# Print network statistics for true and reconstructed network
recon_network_stats = collectStatistics( network_name, method )

In [6]:
print("* Relative Adjacency Error: {}".format(recon_network_stats['rank'+str(rank)]['frobenius']))
print("* Triangles, Original: {}, Reconstructed: {}".format(recon_network_stats['true']['triangles'], recon_network_stats['rank'+str(rank)]['triangles']))
print("* Average Path Length, Original: {}, Reconstructed: {}".format(recon_network_stats['true']['apl'], recon_network_stats['rank'+str(rank)+'bin']['apl']))
print("* Conductance of Ground-Truth Communities")
for community, conductance in recon_network_stats['true']['conductance'].items():
    print("Community Index: {}".format(community))
    recon_conductance = recon_network_stats['rank'+str(rank)]['conductance'][community]
    print("Conductance(True Network): {}, Conductance(Reconstructed Network): {}".format(conductance, recon_conductance))

* Relative Adjacency Error: 0.7534720779221654
* Triangles, Original: 41976, Reconstructed: 58942
* Average Path Length, Original: 2.122629202207727, Reconstructed: 2.112393376818866
* Conductance of Ground-Truth Communities
Community Index: 0
Conductance(True Network): 0.0956274180597124, Conductance(Reconstructed Network): 0.11597128843261491
Community Index: 1
Conductance(True Network): 0.09573224203449283, Conductance(Reconstructed Network): 0.11604234880676369


## Node Classification

In [13]:
labels = N.getLabels()
labels[np.all(labels, axis=0)]

# And low-rank dimensional embedding
filename = os.path.splitext( network_name )[0]
folder = 'adj_recon/'
reconstructed_network = folder +  filename + '_' + str(rank) +'_recon_elts.npy'
Recon = Reconstructed( *N.getNodesVolume() )
Recon.loadNetwork( reconstructed_network, method )
Recon.netmf_embedding( window, skip_max )
Recon.low_rank_embedding( rank )

for train_ratio in [0.1,0.9]:
    print(" -- Train Ratio: {} --".format(train_ratio))
    print("True Network Micro-F1:")
    print(predict_cv(N.LR_Embedding, labels, train_ratio, n_splits=10, random_state=0, C=1.)[0])
    print("Reconstructed Network Micro-F1:")
    print(predict_cv(Recon.LR_Embedding, labels, train_ratio, n_splits=10, random_state=0, C=1.)[0])

 -- Train Ratio: 0.1 --
True Network Micro-F1:
100.0
Reconstructed Network Micro-F1:
100.0
 -- Train Ratio: 0.9 --
True Network Micro-F1:
100.0
Reconstructed Network Micro-F1:
100.0
